# Deep learning
Ce GitHub propose quelques exemples de réseaux de neurones assez simples. Illustrés à travers des exemples (collections de textes) (pris de la litérature: site de Keras ou Tensorflow). Les premiers exemples exloitet ds réseaux multicouhces. Si on arrive à e

## Premiers modèles multicouches

In [ ]:
# Create your first MLP in Keras
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation
from sklearn.model_selection import train_test_split

# fix random seed for reproducibility
seed=7
np.random.seed(7)

# Je télécharge ce dataset que l'on eut trouver partut sur es sites cités ci-dessus. 
path='/Users/boughanem/Pgmes/data/india/pima-indians-diabetes.data.csv'

# load pima indians dataset
dataset = np.loadtxt(path, delimiter=",")
X = dataset[:,0:8]
Y = dataset[:,8]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=seed)

# create model / création du modèle 
# Créer la première couche Cachéé avec 12 neurines et reçoit 8 signaux (entrée) 

model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# La liste des fonctions d'actions proposées par Keras:
#https://keras.io/api/layers/activations/
    
## Le modèle peut être défini comme ceci #######
#model = Sequential([
#    Dense(12, input_dim=8), Activation('relu'),
#    Dense(8),Activation('relu'),
#    Dense(1),Activation('sigmoid')
#])
############
model.summary()


# Compile model : permet de préparer le réseau (le modèle avec ses optmiseurs, ...)
# La liste des optimseurs et fonctions Loss: “https://keras.io/losses, https://keras.io/optimizers”

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model : entrainement du modèle, batch : nombre d'exemples à évaluer avant de moifier les poids
#epoch : le nombre de fois que l'on repasse les les exemples
model.fit(X_train, y_train, epochs=10, batch_size=10)

# evaluate the model, Tester le modèle
scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
model.save("my_first_DL_model.h5")

In [ ]:
# Je télécharge le modèle.
model = keras.models.load_model("my_first_DL_model.h5")

## On reprend le premier exemple de réseaux MLP on utilise Tensorflow

### Un premier exemple ou les données sont déjà pré-traitées l'appel de eras.datasets.imdb.load_data fait le nécessaire. 
La suite consiste juste construire le modèle, puis l'entrainer  , etc.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, datasets
from keras.preprocessing import sequence

# load the dataset but only keep the top n words, zero the rest
top_words = 20000
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data(num_words=top_words)

X_train[0:,]
#chaque commentaire sera représenté par un vecteur de 500 mts.
max_words = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

# create the model

model = keras.Sequential()
model.add(layers.Embedding(top_words, 32, input_length=max_words))
model.add(layers.Flatten())
model.add(layers.Dense(250, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
# Fit the model

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=1, batch_size=128)

# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))


## Dans ce second exemple, on part "from scratch" on part de textes bruts
on fait tous les traitements pour une analyse de sentiment

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import preprocessing
import os

dataset_dir='/Users/boughanem/Pgmes/data/sentiment/aclImdb'
train_dir=os.path.join(dataset_dir, 'train')
test_dir=os.path.join(dataset_dir, 'test')
os.listdir(train_dir)

# utiliser une tf.data.Dataset un ensemble d'outils  pour gérer des données
# En machine learning on a besoin de trois ensmbles : un train, validation et test
# ImDB possède deux sets (train et tesst), on construit un ensemble de validation à partir du Train.
batch_size = 32
seed = 42

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
        train_dir, batch_size=batch_size, validation_split=0.2,subset='training', seed=seed)

#On fait le même traitement pour toutes les données validation et test

# L'ensemble pour la validation
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    train_dir, batch_size=batch_size, validation_split=0.2, subset="validation", seed=seed)

# le test
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    test_dir, batch_size=batch_size
)


# affichage des données de raw_trains_ds, objet de type tf.dataset
for text_batch, label_batch in raw_train_ds.take(1):
  for i in range(2):
    print("Review", text_batch.numpy()[i])
    print("Label", label_batch.numpy()[i])

## Les labels 1 ou 0 sont données en fonction du répértoire
# il y a deux répértoires pos (devrait avoir 1) et neg (devrait avoir 0)

print("Label 0 corresponds to", raw_train_ds.class_names[0])
print("Label 1 corresponds to", raw_train_ds.class_names[1])



Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.
Review b'"Pandemonium" is a horror movie spoof that comes off more stupid than funny. Believe me when I tell you, I love comedies. Especially comedy spoofs. "Airplane", "The Naked Gun" trilogy, "Blazing Saddles", "High Anxiety", and "Spaceballs" are some of my favorite comedies that spoof a particular genre. "Pandemonium" is not up there with those films. Most of the scenes in this movie had me sitting there in stunned silence because the movie wasn\'t all that funny. There are a few laughs in the film, but when you watch a comedy, you expect to laugh a lot more than a few times and that\'s all this film has going for it. Geez, "Scream" had more laughs than this film and that was more of a horror film. How bizarre is that?<br /><br />*1/2 (out of four)'
Label 0
Review b"David Mamet is a very intere

### Préparation des textes: tockenisation, vectorisation , ... à l'aide preprocessing.TextVectorization.
Le texte que l'on traite comporte des données balise de tpe \br non traitées par 
le module de vectorisation standard, on écrit une fonction custom_standardization(input_data) qui nettoie ces données
si votre texte n'a pas de balises utilisez directement la fonction standard (propoée par défaut)

In [2]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import string
import re

## Préparation des textes: tockenisation, vectorisation , ... 
# à laide preprocessing.TextVectorization 
# Reager les données puis extraie les mots 

#cette fonction permet de nettoyer le texte (supprimer les balisesbr, split)
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    return tf.strings.regex_replace(
        stripped_html, "[%s]" % re.escape(string.punctuation), ""
    )

# On vectorise le texte. On utilise la classe TestVectorization qui fait plusieurs opérations.
# lémmatise, split et map chaque mots en un id (einteger) grace à output_mode=int, 
# on peut ussi limter le nombre de mots àprendre en compte dans le dictonnare 20000;
# la taille de l'embedding et la taille de la séquence de text (le vecteur d'entrées, je prends 500 mots)

# Model constants.
max_features = 20000
embedding_dim = 128
sequence_length = 500

vectorize_layer = TextVectorization(
    standardize=custom_standardization, # fonction créé ci-dessus, voir la classe pour les détails
    max_tokens=max_features,
    output_mode="int", # on donne aussi une taille de la séquence, ici on construit un vecteur
                        # le mode binary permet un sac de mot
    output_sequence_length=sequence_length,
)


# on appelle la fonction adapt en donnant le texte pour créer le vocabulaire 
# En fait map prend l'entrée qui est de la forme x, y (text, label), et garde uniquement x (le text)

train_text = raw_train_ds.map(lambda x, y: x)
#la fonction adpat est importante, permet de construire réer un index des mots vers des entiers
vectorize_layer.adapt(train_text)


def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label


# Sélectionner quelques données 
text_batch, label_batch = next(iter(raw_train_ds))
first_review, first_label = text_batch[0], label_batch[0]
print("Review", first_review)
print("Label", raw_train_ds.class_names[first_label])
print("Vectorized review", vectorize_text(first_review, first_label))


Review tf.Tensor(b'Silent Night, Deadly Night 5 is the very last of the series, and like part 4, it\'s unrelated to the first three except by title and the fact that it\'s a Christmas-themed horror flick.<br /><br />Except to the oblivious, there\'s some obvious things going on here...Mickey Rooney plays a toymaker named Joe Petto and his creepy son\'s name is Pino. Ring a bell, anyone? Now, a little boy named Derek heard a knock at the door one evening, and opened it to find a present on the doorstep for him. Even though it said "don\'t open till Christmas", he begins to open it anyway but is stopped by his dad, who scolds him and sends him to bed, and opens the gift himself. Inside is a little red ball that sprouts Santa arms and a head, and proceeds to kill dad. Oops, maybe he should have left well-enough alone. Of course Derek is then traumatized by the incident since he watched it from the stairs, but he doesn\'t grow up to be some killer Santa, he just stops talking.<br /><br />T

### Vectorisation de toutes les données 

In [3]:
# Vectorize the data.
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)


## Définiton du modèle:
La première couche Embedding prend un vecteur d'entiers (de mots), chaque entie est représentés par un vecteur embedding de 128 éléments. On passe ensuite dans un Dropout, puis une couche dense de 100 neuones, puis une un Dropot puis une sortie un neurone. 

### Tester avec un seul modèle à la fois

In [ ]:
# On construit le modèle
#MLP

model = tf.keras.Sequential([
  layers.Embedding(max_features + 1, embedding_dim),
  layers.Dropout(0.2),
  layers.Dense(100, activation="relu"),
  layers.Dropout(0.3), 
  layers.Dense(1, activation="softmax")])

model.summary()

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# enrinement
model.fit(train_ds, validation_data=val_ds, epochs=3)

#evaluation
model.evaluate(test_ds)



### Une autre façon de définir le modèle est de le voir comme f(g(w(..(x))))
x étant l'entrée et les f,g, w dont des dess transformations (couches du réseau)

In [5]:
# Une autre façon de définir le modèle est de le voir comme f(g(w(..(x))))
# x étant l'entrée et les f,g, w les transformations

# l'entrée est un vecteurs d'entiers
inputs = tf.keras.Input(shape=(None,), dtype="int64")

# chaque entrée est ensuite mappée en un vecteur embeddings : max_featutes de embeddings
x = layers.Embedding(max_features, embedding_dim)(inputs)
x = layers.Dropout(0.2)(x)
x = layers.Dense(100, activation="relu")(x)
x = layers.Dropout(0.3)(x)

# on projette sur ladernière couche 1 neurone 
predictions = layers.Dense(1, activation="softmax", name="predictions")(x)

model = tf.keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


# Training
model.fit(train_ds, validation_data=val_ds, epochs=3)


#evaluation
model.evaluate(test_ds)

Epoch 1/3
625/625 [==============================] - 37s 58ms/step - loss: 7.6992 - accuracy: 0.4951 - val_loss: 7.5087 - val_accuracy: 0.5076
Epoch 2/3
625/625 [==============================] - 37s 59ms/step - loss: 7.7024 - accuracy: 0.4949 - val_loss: 7.5087 - val_accuracy: 0.5076
Epoch 3/3
782/782 [==============================] - 12s 15ms/step - loss: 7.6246 - accuracy: 0.5000


[7.624587535858154, 0.5]

## Faire une prédiction à partir d'un texte externe,

In [8]:
# dans nore modèle la vectorisation des textes, s'est faite à l'extérieur du modèle.
# ceci permet un traitement rapide.

examples = [
  "The movie was great!",
  "The movie was okay.",
  "The movie was terrible..."
]

print(examples[0])
examples_vec = vectorize_layer(examples)
model.predict(examples_vec)


The movie was great!


array([[[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]],

       [[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]],

       [[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]]], dtype=float32)

## La prédiction 
Le mieux est de construire un modèle (exporter le modèle appris vers un modèle nouveau)

In [12]:
export_model = tf.keras.Sequential([
  vectorize_layer,
  model,
  layers.Activation('sigmoid')
])

export_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [10]:
examples = [
  "The movie was great!",
  "The movie was okay.",
  "The movie was terrible..."
]


export_model.predict(examples)

array([[[0.7310586],
        [0.7310586],
        [0.7310586],
        ...,
        [0.7310586],
        [0.7310586],
        [0.7310586]],

       [[0.7310586],
        [0.7310586],
        [0.7310586],
        ...,
        [0.7310586],
        [0.7310586],
        [0.7310586]],

       [[0.7310586],
        [0.7310586],
        [0.7310586],
        ...,
        [0.7310586],
        [0.7310586],
        [0.7310586]]], dtype=float32)

### On peut définir un modèle avec la vectorisation comme partie du modèle.
Peut être couteux à éviter

In [ ]:
# on peut définir un modèle avec la vectorisation comme faisant partie du modèle
# Le traitement sera plus long

text_input = tf.keras.Input(shape=(1,), dtype=tf.string, name='text')
x = vectorize_layer(text_input)
x = layers.Embedding(max_features + 1, embedding_dim)(x)

x = layers.Dropout(0.2)(x)
x = layers.Dense(100, activation="relu")(x)
x = layers.Dropout(0.3)(x)

# on projette sur la dernière couche 1 neurone 
predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)

model_vec = tf.keras.Model(text_input, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model_vec.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Training
model_vec.fit(raw_train_ds, validation_data=raw_val_ds, epochs=3)


#evaluation
model_vec.evaluate(raw_test_ds)

## 2. Modèle à Convolution (CNN)

Ce type de modèle utilisé pour les images (le premier exemple donneu exemple de recherche d'image à partir d'u dataset cifar10).
Le second exemple exploite un CNN sur notre texte.
### 1 Exemple classification d'images

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

import matplotlib.pyplot as plt

(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()


model = Sequential()
# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)

#model.fit(x_train, y_train, batch_size=32, epochs=10)
model.summary()


## 2 Exemple : un modèle CNN sur notre base de textes
on utilise les versions vectorisées dans nos textes

In [4]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

# A integer input for vocab indices.
inputs = tf.keras.Input(shape=(None,), dtype="int64")

# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'.
x = layers.Embedding(max_features, embedding_dim)(inputs)
x = layers.Dropout(0.5)(x)

# Conv1D + global max pooling
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

# We add a vanilla hidden layer:
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

# We project onto a single unit output layer, and squash it with a sigmoid:
predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)

model_cnn = tf.keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model_cnn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

epochs = 20

# Fit the model using the train and test datasets.
model_cnn.fit(train_ds, validation_data=val_ds, epochs=epochs)

# evaluate
model_cnn.evaluate(test_ds)

Epoch 1/20
625/625 [==============================] - 47s 74ms/step - loss: 0.6371 - accuracy: 0.5727 - val_loss: 0.3262 - val_accuracy: 0.8620
Epoch 2/20
625/625 [==============================] - 50s 80ms/step - loss: 0.2908 - accuracy: 0.8843 - val_loss: 0.3114 - val_accuracy: 0.8782
Epoch 3/20
625/625 [==============================] - 47s 75ms/step - loss: 0.1371 - accuracy: 0.9504 - val_loss: 0.4469 - val_accuracy: 0.8804
Epoch 4/20
625/625 [==============================] - 46s 73ms/step - loss: 0.0659 - accuracy: 0.9768 - val_loss: 0.5537 - val_accuracy: 0.8654
Epoch 5/20
625/625 [==============================] - 45s 73ms/step - loss: 0.0331 - accuracy: 0.9896 - val_loss: 0.5636 - val_accuracy: 0.8808
Epoch 6/20
625/625 [==============================] - 45s 73ms/step - loss: 0.0307 - accuracy: 0.9884 - val_loss: 0.6306 - val_accuracy: 0.8754
Epoch 7/20
625/625 [==============================] - 46s 73ms/step - loss: 0.0242 - accuracy: 0.9912 - val_loss: 0.6398 - val_accuracy:

[1.3953273296356201, 0.8496400117874146]

## De même pour prédire un texte externe (à la base de test)
On crée un oveau modèle on ajoute juste la vectorisation des entrées.
le modèle utilise les poids appris précédemment.

In [5]:

# A string input
inputs = tf.keras.Input(shape=(1,), dtype="string")
# Turn strings into vocab indices
indices = vectorize_layer(inputs)
# Turn vocab indices into predictions
outputs = model_cnn(indices)

# Our end to end model
end_to_end_model = tf.keras.Model(inputs, outputs)

end_to_end_model.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]
)


In [6]:
# Test it with `raw_test_ds`, which yields raw strings
examples = [
  "The movie was great!",
  "The movie was okay.",
  "The movie was terrible..."
]

print(examples[0])


end_to_end_model.predict(examples)


The movie was great!


array([[0.6432649 ],
       [0.3580997 ],
       [0.13735926]], dtype=float32)